In [ ]:

import wandb
wandb.login()

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV   
# %pip install ydata-profiling
# %pip install ipywidgets
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score

from sklearn.ensemble import RandomForestClassifier

# Iniciar sesión en Wandb



# Initialize Wandb project
wandb.init(project="model-comparison", entity="wandb")

WANDB_NOTEBOOK_NAME = "model-comparison"

# export WANDB_INIT_TIMEOUT= 120
#exportar wandb init timeout para evitar errores de tiempo de espera




In [ ]:
df = pd.read_csv('./data.csv')

In [ ]:
df.drop(columns=df.columns[0], axis=1, inplace=True)

In [ ]:
y = df['group']
X = df.drop(['group', 'Initial_EDSS','Final_EDSS'], axis=1)
#dropear columna Initial_EDSS y Final_EDSS
df.drop(['Initial_EDSS','Final_EDSS'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.5, random_state=100)

# Dividir X_temp y y_temp en conjuntos de validación y prueba
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=100)


In [ ]:
x_train_clean = X_train.dropna()
y_train_clean = y_train.loc[x_train_clean.index]

x_test_clean = X_test.dropna()
y_test_clean = y_test.loc[x_test_clean.index]

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [2, 4, 6, 8, 10, 12, None],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
    "criterion": ["gini", "entropy"]
     
}


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth=2, random_state=100)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(x_train_clean, y_train_clean)


best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

In [ ]:
wandb.config.update(best_params)

# Evaluate on test set
y_pred = best_model.predict(x_test_clean)
accuracy = accuracy_score(y_test_clean, y_pred)

# Log accuracy to Wandb
wandb.log({"Test Accuracy": accuracy})



In [ ]:
# Calculate ROC AUC
y_prob = best_model.predict_proba(x_test_clean)[:, 1]
auc = roc_auc_score(y_test_clean, y_prob)
wandb.log({"ROC AUC": auc})

# Plot ROC curve
fpr, tpr, _ = roc_curve(y_test_clean, y_prob)
plt.plot(fpr, tpr, label="ROC curve (area = %0.2f)" % auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate') 

plt.title('Receiver operating characteristic')
wandb.log({"ROC Curve": plt})

In [ ]:
# Finish Wandb run
wandb.finish()

In [ ]:
df.head(144)

In [ ]:
print("Accuracy:", accuracy)
print("Recall:", recall)
print("F1-score:", f1)
print("AUC:", auc)

## WANDB